In [1]:
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import pickle
import numpy as np
from collections import OrderedDict
from common.layers import *
from common.gradient import numerical_gradient

In [2]:
class SimpleConvNet:
    def __init__(self, input_dim=(1,28, 28),  #input_dim : (채널수, 높이, 너비)의 차원
                conv_param={'filter_num':30, 'filter_size':5,
                           'pad':0, 'stride':1},
                hiddden_size=100, output_size=10, weight_init_std=0.01): #hidden_size : 은닉층(완전연결)의 뉴런 수, output_size : 출력층(완전연결)의 뉴런수
        
        ## 초기화 인수, 주어진 합성곱 계층의 하이퍼파라미터를 딕셔너리에서 꺼냄
        filter_num=conv_param['filter_num']
        filter_size=conv_param['filter_size']
        filter_pad=conv_param['stride']
        input_size=input_dim[1]
        conv_output_size=(input_size-filter_size+2*filter_pad)/filter_stride+1
        pool_output_size=int(filter_num*(conv_output_size/2)*(conv_output_size/2))
        
        ## 가중치 매개변수를 초기화
        self.params={}
        self.params['W1']=weight_init_std*np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1']=wieght_init_std*np.random.randn(pool_output_size, hidden_size)
        
        self.params['W2']=wieght_init_std*np.random.randn(pool_output_size, hidden_size)
        self.params['b2']=np.zeros(hidden_size)
        
        self.params['W3']=weight_init_std*np.random.randn(hidden_size, output_size)
        self.params['b3']=np.zeros(output_size)
        
        ## 계층을 생성
        self.layers=OrderedDict()
        self.layers['Conv1']=Convolution(self.params['W1'], self.params['b1'], conv_param['stride'], conv_param['pad'])
        self.layers['Relu1']=Relu()
        self.layers['Pool1']=Pooling(pool_h=2, pool_w=2, stride=2)
        
        self.layers['Affine1']=Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2']=Relu()
        
        self.layers['Affine2']=Affine(self.params['W3'], self.params['b3'])

        self.last_layer=SoftmaxWithLoss()
    
    ## forward 함수
    def predict(self, x):
        for layer in self.layers.values():
            x=layer.forward(x)
        return x #softmax에 넣기 전의 forward 결과값이 나옴
    
    ## 손실을 구하는 함수
    def loss(self, x, t):
        """손실 함수를 구한다.

        Parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블
        """
        y=self.predict(x)
        return self.last_layer.forward(y,t)
    
    #정확도를 구하는 함수
    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1:t=np.argmax(t, axis=1)
        
        acc=0.0
        
        for i in range(int(x.shape[0]/batch_size)):
            tx=x[i*batch_size:(i+1)*batch_size]
            tt=t[i*batch_size:(i+1)*batch_size]
            y=self.predict(tx)
            y=np.argmax(y, axis=1)
            acc += np.sum(y==tt)
            
        return acc/x.shape[0]
    
    ##수치미분으로 기울기를 구하는 함수
    def numerical_gradient(self, x, t):
        loss_w=lambda w: self.loss(x,t)
        
        grads={}
        for idx in (1,2,3):
            grads['W'+str(idx)]=numerical_gradient(loss_w, self.params['W'+str(idx)])
            grads['b'+str(idx)]=numerical_gradient(loss_w, self.params['b'+str(idx)])
        return grads
    
    ##오차역전파법으로 기울기를 구하는 함수
    def gradient(self, x, t):
        #순전파
        self.loss(x,t)
        
        #역전파
        dout=1
        dout=self.last_layer.backward(dout)
        
        layers=list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout=layer.backward(dout)
            
        #결과 저장
        grads={}
        grads['W1']=self.layers['Conv1'].dW
        grads['b1']=self.layers['Conv1'].db
        grads['W2']=self.layers['Affine1'].dW
        grads['b2']=self.layers['Affine1'].db
        grads['W3']=self.layers['Affine2'].dW
        grads['b3']=self.layers['Affine2'].db
        
        return grads
    
    #parameter를 저장하는 함수
    def save_params(self, file_name="params.pkl"):
        params={}
        for key, val in self.params.items():
            params[key]=val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)
    
    #parameter를 load하는 함수
    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params=pickle.load(f)
        for key, val in paras.items():
            self.params[key]=val
        
        for i, key in enumerate(['Conv1', 'Affine1', 'Affine2']):
            self.layers[key].W=self.params['W'+str(i+1)]
            self.layers[key].b=self.params['b'+str(i+1)]
        
    